## 步骤 1 - 安装所需的 Python 环境及包

## 步骤 2 - 初步读入并清洗数据

In [ ]:
import pandas as pd

df = pd.read_csv("./tweets.csv")
df.head(20)

我们可以发现，Roberta 使用 0/1/2 来分别代表 negative/neutral/positive，而 gpt2 使用 LABEL_0/1/2 来代表 negative/neutral/positive。为了方便后续处理，我们需要进行清理。

In [ ]:
def label_map_gpt2(x):
    #TODO
    if x == "LABEL_0":
        x = "negative"
    elif x == "LABEL_1":
        x = "neutral"
    elif x == "LABEL_2":
        x = "positive"
    else:
        x = "invalid"
    return x
    
def label_map_roberta(x):
    #TODO
    if x == "0":
        x = "negative"
    elif x == "1":
        x = "neutral"
    elif x == "2":
        x = "positive"
    else:
        x = "invalid"
    return x

df['gpt2'] = df['gpt2'].map(label_map_gpt2)
df['roberta'] = df['roberta'].map(label_map_roberta)
df.head(20)

Zeno 处理要求我们增加 input_length 列与 id 列，对数据进行进一步处理

In [ ]:
df["input_length"] = df["text"].str.len()
df['id'] = df.index
df.head(20)

## 步骤 3 - 启动 Zeno 进行模型分析

创建一个 [Zeno](https://hub.zenoml.com/account) 账号，阅读如下代码并正确运行，运行完成后你将在个人账户下看到创建的 projects

In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

client = ZenoClient("zen_kkuXl7D7TdHaNNVySJFS-NBTKNCff0DvSd-JGjeICbI")

# 创建项目
proj = client.create_project(
    name="Twitter Sentiment Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="roberta_accuracy", type="mean", columns=["roberta_correct"]),
        #TODO 类比上述方法，为 gpt2 模型创建相应的评估指标
        ZenoMetric(name="gpt2_accuracy", type="mean", columns=["gpt2_correct"])
    ]
)

proj.upload_dataset(df, id_column="id", data_column='text', label_column="label")

# 为 Roberta 模型分别创建系统数据框
df_roberta = pd.DataFrame({
    "id": df["id"],
    "output": df["roberta"],
    "roberta_correct": (df["roberta"] == df["label"]).astype(int)
})
proj.upload_system(df_roberta, name="Roberta", id_column="id", output_column="output")
#TODO 类比上述方法，为 gpt2 模型创建系统数据框
df_gpt2 = pd.DataFrame({
    "id": df["id"],
    "output": df["gpt2"],
    "gpt2_correct": (df["gpt2"] == df["label"]).astype(int)
})
proj.upload_system(df_gpt2, name="gpt2", id_column="id", output_column="output")


## 步骤 4 - 创建分析切片并进一步分析

你一共需要利用 Zeno 提供的接口创建 5 个不同的切片。

你可以首先创建两个切片：

1. 有标签的推文（含有"#"）
- 表现特点：标签内容会对模型输出产生比较大的影响。准确率和全部数据集上相近（gpt2略高，roberta略低）。
2. 有强烈正向情感的词语（如 love），你可以自行决定一个词
- 表现特点：模型输出强烈倾向于"positive"，少数情况是"netrual"，不会出现"negative"。准确率显著高于全部数据集上。

创建切片可以直接通过点击 "+" 按钮来进行，可以通过基本值匹配或正则表达式的方式创建，具体用法参见[文档](https://zenoml.com/docs/intro/)

![image.png](image.png)

有关 Zeno 中更多有趣的用法可以参考 Zeno 仓库中的 [README](https://github.com/zeno-ml/zeno)。

在此处写下你想要另外创建的三个切片，并在个人 Zeno 账户中创建，针对每一个创建的切片，用一两句话总结模型在其上的表现特点：

3. 有不可阅读的符号的推文（含有“”）
- 表现特点：数据集中label有问题，gpt2存在输出非label的情况（标记为“invalid”）。准确率低于整体数据集上，数据集也出了些问题。
4. 长度小于50的推文
- 表现特点：准确率显著高于整体数据集。模型似乎更倾向于"positive"和"negative"
5. 长度大于120的推文
- 表现特点：准确率低于整体数据集。

## 提交：
1. 本 notebook，保留你书写的代码与输出结果
2. 一张截图，显示你创建的 5 个切片